In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import pickle
import json
import cv2
import re

In [2]:
tf.test.is_gpu_available()

True

In [3]:
# make image dataloader using flow_from_dataframe
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# load data to extract labels
data_dir = '../facebook_challenge_data/'
model_dir = 'models/'

# load data and print sizes
def get_dict(path):
    jsonl_content = open(path, 'r').read()
    data = [json.loads(jline) for jline in jsonl_content.split('\n')]
    return {datum['id'] : datum for datum in data}


train_dict = get_dict(data_dir + 'train.jsonl')
val_dict = get_dict(data_dir + 'dev.jsonl')
test_dict = get_dict(data_dir + 'test.jsonl')

print(len(train_dict))
print(len(val_dict))
print(len(test_dict))

8500
500
1000


In [18]:
# custom data generator to handle cropping
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
from random import randint # for random cropping

class FBDataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data_dict, batch_size=32, dim=(299, 299), n_channels=3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.data_dict = data_dict
        self.shuffle = shuffle
        
        # build labels list and id list
        self.id_list = list(self.data_dict.keys())
        self.labels = {ID: self.data_dict[ID]['label'] for ID in self.id_list}
        self.img_list = {ID: self.data_dict[ID]['img'] for ID in self.id_list}
            
        self.on_epoch_end()
        self.classes = [self.labels[self.id_list[i]] for i in self.indexes]

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.id_list) / self.batch_size)) + 1 # last batch is partial

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:index*self.batch_size + self.batch_size]
        
        
        # Find list of IDs
        id_list_temp = [self.id_list[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(id_list_temp)
        
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.id_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, id_list_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((len(id_list_temp), *self.dim, self.n_channels))
        y = np.empty(len(id_list_temp), dtype=int)

        # Generate data
        for i, ID in enumerate(id_list_temp):
            # Store sample
            X[i,] = self.process_img(data_dir + self.img_list[ID])

            # Store class
            y[i] = self.labels[ID]

        return X, y
    
    def process_img(self, path): # method for getting image
        img = Image.open(path)
        img.load()
        img = img.resize((int(dim * 1.5) for dim in self.dim), Image.ANTIALIAS)
        data = np.asarray(img, dtype='uint8')
        im = self.augment(data)
        
        
        if im.shape==(self.dim[0], self.dim[1]): im = np.stack((im,)*3, axis=-1) # handle grayscale
        if im.shape == (*self.dim, 4): im = im[:,:,:3] # handle weird case
        
        return im
    
    def augment(self, im): # random crop and random mirror
        
        # random crop
        x_max, y_max = im.shape[0], im.shape[1]
        x_start, y_start = randint(0, x_max - self.dim[0]), randint(0, y_max - self.dim[1])
        im = im[x_start:x_start + self.dim[0], y_start:y_start + self.dim[1]]
        
        # random mirror
        if randint(0,1): im = np.flip(im, axis=1)
        
        return im


In [19]:
# create data generators
train_gen = FBDataGenerator(data_dict=train_dict,
                          batch_size=32,
                          dim=(299, 299),
                          n_channels=3,
                          shuffle=True)

val_gen = FBDataGenerator(data_dict=val_dict,
                          batch_size=32,
                          dim=(299, 299),
                          n_channels=3,
                          shuffle=True)

In [20]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten

conv_base = tf.keras.applications.inception_v3.InceptionV3(include_top=False, 
                                                        weights='imagenet', 
                                                        input_shape=(299, 299, 3))
for layer in conv_base.layers[:-1]: layer.trainable = False # freeze pretrained layers

model = Sequential()
model.add(conv_base)
model.add(Flatten())
# model.add(Dense(2048, activation='relu'))
# model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

optimizer = Adam(lr = 1e-6)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 8, 8, 2048)        21802784  
_________________________________________________________________
flatten_3 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               67109376  
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 513       
Total params: 88,912,673
Trainable params: 67,109,889
Non-trainable params: 21,802,784
_________________________________________________________________
None


In [21]:
# train model

# from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

# model = load_model(model_dir + 'cnn_weighted.h5')

mcp_save = ModelCheckpoint(model_dir + 'best_fb_inc_cnn.h5', save_best_only=True, monitor='val_loss', mode='min')

history = model.fit_generator(train_gen,
                    validation_data=val_gen,
                    shuffle=True,
                    epochs=25,
                    callbacks=[mcp_save])


Epoch 1/25
266/266 [==============================] - 377s 1s/step - loss: 0.7014 - accuracy: 0.5971 - val_loss: 7.1687 - val_accuracy: 0.5020
Epoch 2/25
266/266 [==============================] - 374s 1s/step - loss: 0.6760 - accuracy: 0.6133 - val_loss: 7.2236 - val_accuracy: 0.5040
Epoch 3/25
266/266 [==============================] - 378s 1s/step - loss: 0.6573 - accuracy: 0.6287 - val_loss: 7.0406 - val_accuracy: 0.5120
Epoch 4/25
266/266 [==============================] - 390s 1s/step - loss: 0.6488 - accuracy: 0.6351 - val_loss: 6.8386 - val_accuracy: 0.5100
Epoch 5/25
266/266 [==============================] - 377s 1s/step - loss: 0.6494 - accuracy: 0.6364 - val_loss: 6.5251 - val_accuracy: 0.4940
Epoch 6/25
266/266 [==============================] - 363s 1s/step - loss: 0.6371 - accuracy: 0.6436 - val_loss: 6.8222 - val_accuracy: 0.4840
Epoch 7/25
266/266 [==============================] - 363s 1s/step - loss: 0.6359 - accuracy: 0.6431 - val_loss: 7.3592 - val_accuracy: 0.5040

In [22]:
val_gen = FBDataGenerator(data_dict=val_dict,
                          batch_size=32,
                          dim=(299, 299),
                          n_channels=3,
                          shuffle=True)

In [23]:
# test
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import math

y_val = val_gen.classes

# get AUROC
preds = model.predict_generator(val_gen)
print('Test AUROC:', roc_auc_score(y_val, preds))

# get loss and acc
preds_bin = np.array(preds)
preds_bin[preds>0.5] = 1
preds_bin[preds<=0.5] = 0
print('Test Accuracy:', accuracy_score(y_val, preds_bin))

# get F1
print('Test F1:', f1_score(y_val, preds_bin, zero_division=1))
print('Test Precision:', precision_score(y_val, preds_bin, zero_division=1))
print('Test Recall:', recall_score(y_val, preds_bin, zero_division=1))

Test AUROC: 0.48412
Test Accuracy: 0.492
Test F1: 0.29050279329608936
Test Precision: 0.48148148148148145
Test Recall: 0.208


In [ ]:
# model.save(model_dir + 'cnn_weighted.h5')

In [ ]:
# get loss and acc with lower threshold
preds_bin = np.array(preds)
preds_bin[preds>0.4] = 1
preds_bin[preds<=0.4] = 0
print('Test Accuracy:', accuracy_score(y_test, preds_bin))

# get F1
print('Test F1:', f1_score(y_test, preds_bin, zero_division=1))
print('Test Precision:', precision_score(y_test, preds_bin, zero_division=1))
print('Test Recall:', recall_score(y_test, preds_bin, zero_division=1))

In [ ]:
from tensorflow.keras.models import load_model

model = load_model(model_dir + 'cnn_weighted.h5')

In [ ]:
preds = model.predict_generator(test_gen)
y_test = np.array(test_gen.get_labels())

pos_preds = preds[y_test == 1]
neg_preds = preds[y_test == 0]

In [ ]:
plt.title('Distribution of Prediction Values (Image Only) (Full CNN with 1:1/0.28 weighting)\n 8 epochs 1e-6 lr')
plt.hist(pos_preds, bins=100, alpha=0.5, label='positive')
plt.hist(neg_preds, bins=100, alpha=0.5, label= 'negative')
plt.legend()

In [9]:
for im in os.listdir('../facebook_challenge_data/img/'):
    img = Image.open('../facebook_challenge_data/img/' + im)
    print(img.size)

(240, 400)
(550, 366)
(550, 366)
(800, 800)
(550, 398)
(550, 404)
(550, 366)
(313, 399)
(825, 549)
(297, 400)
(266, 400)
(550, 366)
(550, 412)
(825, 549)
(800, 533)
(550, 366)
(825, 549)
(267, 400)
(550, 367)
(825, 543)
(825, 551)
(825, 538)
(672, 800)
(550, 366)
(825, 549)
(800, 533)
(282, 400)
(305, 400)
(550, 366)
(550, 350)
(531, 800)
(538, 800)
(550, 366)
(550, 366)
(240, 400)
(825, 549)
(825, 549)
(550, 374)
(800, 533)
(543, 800)
(565, 800)
(310, 400)
(800, 532)
(800, 533)
(393, 800)
(825, 575)
(550, 412)
(533, 800)
(550, 366)
(533, 800)
(550, 366)
(800, 533)
(550, 378)
(573, 800)
(550, 366)
(266, 400)
(825, 549)
(550, 365)
(800, 533)
(277, 399)
(266, 400)
(550, 366)
(550, 366)
(550, 366)
(550, 422)
(800, 533)
(294, 400)
(550, 366)
(550, 412)
(800, 523)
(550, 410)
(825, 549)
(696, 800)
(825, 631)
(724, 800)
(533, 800)
(825, 634)
(550, 366)
(550, 366)
(417, 800)
(800, 533)
(550, 366)
(480, 800)
(550, 435)
(300, 400)
(550, 365)
(225, 400)
(533, 800)
(533, 800)
(750, 800)
(800, 531)

(524, 800)
(550, 435)
(825, 659)
(579, 800)
(550, 366)
(591, 800)
(265, 400)
(825, 816)
(825, 592)
(281, 400)
(800, 800)
(800, 533)
(550, 366)
(323, 400)
(312, 800)
(825, 549)
(550, 366)
(550, 366)
(294, 400)
(268, 400)
(696, 800)
(825, 549)
(550, 366)
(300, 400)
(295, 399)
(245, 400)
(825, 549)
(550, 412)
(291, 400)
(825, 549)
(550, 373)
(550, 366)
(702, 800)
(800, 515)
(800, 533)
(550, 366)
(550, 365)
(550, 365)
(825, 549)
(800, 533)
(800, 533)
(825, 576)
(286, 400)
(600, 800)
(285, 400)
(800, 800)
(550, 366)
(800, 533)
(533, 800)
(825, 549)
(825, 546)
(550, 366)
(825, 539)
(825, 549)
(550, 388)
(266, 400)
(825, 407)
(550, 366)
(825, 549)
(825, 549)
(533, 800)
(550, 480)
(825, 547)
(550, 366)
(550, 366)
(825, 549)
(533, 800)
(321, 400)
(267, 400)
(825, 502)
(637, 800)
(268, 400)
(550, 350)
(277, 399)
(642, 800)
(550, 366)
(559, 800)
(267, 400)
(825, 636)
(825, 549)
(825, 549)
(550, 366)
(800, 533)
(550, 366)
(550, 366)
(400, 400)
(266, 400)
(550, 342)
(550, 516)
(550, 365)
(594, 800)

(266, 400)
(800, 800)
(615, 800)
(550, 366)
(616, 800)
(400, 400)
(550, 340)
(641, 800)
(532, 800)
(550, 366)
(302, 400)
(825, 549)
(326, 800)
(550, 366)
(266, 400)
(591, 799)
(533, 800)
(550, 412)
(296, 400)
(825, 621)
(589, 800)
(800, 525)
(347, 400)
(550, 524)
(825, 546)
(269, 400)
(550, 366)
(280, 400)
(266, 400)
(825, 549)
(666, 800)
(247, 400)
(550, 366)
(533, 800)
(240, 400)
(575, 800)
(550, 384)
(825, 549)
(800, 800)
(825, 547)
(519, 800)
(825, 549)
(800, 533)
(395, 400)
(599, 799)
(550, 366)
(825, 573)
(550, 449)
(550, 466)
(467, 800)
(550, 366)
(825, 549)
(298, 399)
(550, 382)
(550, 403)
(825, 549)
(550, 366)
(160, 400)
(550, 366)
(611, 799)
(550, 366)
(550, 366)
(300, 400)
(589, 800)
(550, 366)
(554, 800)
(200, 400)
(550, 359)
(267, 400)
(240, 400)
(550, 366)
(400, 400)
(550, 412)
(550, 366)
(600, 800)
(800, 800)
(550, 384)
(550, 366)
(550, 366)
(800, 615)
(284, 400)
(532, 800)
(360, 800)
(657, 800)
(267, 400)
(825, 549)
(825, 549)
(825, 539)
(550, 366)
(400, 400)
(118, 400)

(800, 594)
(550, 365)
(800, 628)
(550, 366)
(550, 383)
(550, 366)
(800, 533)
(417, 800)
(825, 549)
(825, 549)
(800, 533)
(550, 365)
(339, 400)
(825, 592)
(800, 533)
(541, 800)
(800, 553)
(800, 533)
(267, 400)
(825, 549)
(550, 439)
(825, 549)
(304, 400)
(550, 368)
(550, 366)
(825, 549)
(550, 357)
(800, 533)
(825, 823)
(825, 549)
(550, 366)
(550, 366)
(764, 800)
(266, 400)
(825, 549)
(825, 551)
(800, 684)
(800, 800)
(550, 368)
(532, 800)
(533, 800)
(825, 575)
(565, 800)
(550, 384)
(825, 435)
(550, 366)
(184, 400)
(800, 531)
(825, 547)
(240, 400)
(550, 425)
(524, 800)
(533, 800)
(550, 366)
(550, 359)
(611, 799)
(825, 549)
(825, 575)
(825, 549)
(825, 549)
(550, 325)
(800, 533)
(550, 366)
(825, 549)
(550, 366)
(825, 549)
(825, 549)
(533, 800)
(825, 731)
(145, 400)
(825, 549)
(550, 366)
(550, 346)
(550, 366)
(533, 800)
(318, 399)
(267, 400)
(825, 533)
(598, 800)
(258, 399)
(569, 800)
(550, 366)
(800, 533)
(825, 549)
(240, 400)
(550, 366)
(825, 549)
(550, 366)
(825, 549)
(825, 549)
(521, 800)

(673, 800)
(400, 400)
(615, 800)
(550, 366)
(517, 799)
(550, 366)
(825, 549)
(800, 544)
(717, 800)
(825, 549)
(825, 464)
(550, 366)
(825, 781)
(825, 547)
(550, 366)
(825, 549)
(550, 366)
(400, 400)
(550, 366)
(300, 400)
(134, 400)
(375, 400)
(550, 366)
(800, 570)
(550, 366)
(266, 400)
(600, 800)
(825, 549)
(550, 366)
(800, 533)
(550, 366)
(259, 400)
(550, 366)
(550, 359)
(825, 547)
(550, 373)
(648, 800)
(550, 365)
(550, 366)
(326, 400)
(267, 400)
(550, 366)
(552, 800)
(605, 800)
(800, 800)
(550, 366)
(598, 800)
(550, 366)
(282, 400)
(550, 485)
(406, 800)
(825, 547)
(550, 366)
(825, 618)
(550, 366)
(825, 549)
(363, 400)
(800, 450)
(825, 549)
(825, 549)
(550, 407)
(550, 469)
(550, 366)
(550, 357)
(550, 412)
(487, 800)
(825, 592)
(313, 399)
(550, 366)
(800, 533)
(550, 370)
(712, 800)
(730, 800)
(550, 366)
(550, 365)
(825, 549)
(825, 549)
(300, 400)
(800, 791)
(550, 366)
(800, 531)
(825, 549)
(269, 400)
(825, 549)
(800, 800)
(550, 366)
(640, 800)
(800, 533)
(550, 365)
(825, 528)
(550, 366)

...

(550, 366)
(825, 549)
(528, 800)
(825, 526)
(825, 533)
(600, 800)
(825, 547)
(273, 400)
(174, 400)
(800, 616)
(825, 549)
(800, 523)
(550, 366)
(800, 533)
(150, 400)
(529, 800)
(800, 639)
(550, 366)
(320, 400)
(825, 549)
(550, 366)
(800, 533)
(825, 520)
(550, 366)
(597, 800)
(800, 533)
(311, 400)
(800, 530)
(825, 631)
(550, 366)
(667, 800)
(266, 400)
(274, 400)
(550, 548)
(550, 366)
(550, 406)
(825, 549)
(550, 382)
(800, 600)
(800, 533)
(800, 533)
(550, 366)
(532, 800)
(825, 549)
(825, 549)
(550, 366)
(550, 366)
(550, 366)
(825, 592)
(245, 400)
(327, 400)
(825, 549)
(341, 400)
(550, 469)
(600, 800)
(825, 546)
(550, 368)
(800, 533)
(825, 515)
(800, 533)
(825, 618)
(550, 366)
(550, 366)
(550, 365)
(825, 549)
(299, 400)
(300, 400)
(533, 800)
(550, 402)
(550, 377)
(825, 601)
(825, 549)
(550, 355)
(539, 800)
(825, 549)
(825, 549)
(550, 366)
(717, 800)
(550, 366)
(800, 640)
(550, 366)
(693, 800)
(550, 366)
(825, 549)
(825, 549)
(550, 365)
(383, 800)
(800, 533)
(550, 365)
(550, 366)
(800, 632)

(550, 366)
(825, 546)
(800, 529)
(550, 367)
(533, 800)
(266, 400)
(787, 800)
(550, 366)
(825, 549)
(800, 533)
(825, 549)
(550, 366)
(550, 355)
(825, 549)
(550, 366)
(800, 533)
(800, 600)
(533, 800)
(544, 800)
(550, 366)
(825, 588)
(825, 549)
(533, 800)
(550, 335)
(800, 601)
(550, 366)
(240, 400)
(530, 799)
(287, 400)
(800, 533)
(825, 528)
(550, 412)
(550, 346)
(550, 327)
(533, 800)
(550, 366)
(267, 400)
(317, 400)
(825, 549)
(550, 366)
(600, 800)
(267, 400)
(407, 800)
(506, 800)
(299, 400)
(800, 533)
(550, 366)
(550, 460)
(133, 400)
(500, 800)
(550, 366)
(550, 431)
(311, 400)
(825, 549)
(825, 549)
(825, 549)
(550, 430)
(533, 800)
(825, 549)
(550, 366)
(550, 366)
(800, 533)
(800, 533)
(800, 800)
(730, 800)
(550, 363)
(550, 366)
(550, 366)
(266, 400)
(550, 366)
(825, 549)
(422, 800)
(552, 799)
(409, 800)
(561, 800)
(134, 400)
(800, 800)
(800, 533)
(396, 400)
(825, 568)
(264, 400)
(533, 800)
(800, 539)
(564, 800)
(550, 414)
(825, 549)
(825, 549)
(825, 549)
(400, 400)
(550, 366)
(600, 800)

(550, 412)
(825, 549)
(550, 344)
(800, 533)
(550, 366)
(800, 533)
(268, 400)
(550, 366)
(550, 366)
(550, 487)
(533, 800)
(825, 447)
(677, 800)
(400, 400)
(550, 366)
(825, 549)
(266, 400)
(825, 549)
(659, 800)
(825, 823)
(825, 552)
(550, 366)
(825, 639)
(825, 551)
(800, 686)
(550, 366)
(800, 526)
(825, 557)
(474, 800)
(550, 366)
(290, 400)
(800, 533)
(550, 366)
(825, 549)
(800, 551)
(550, 272)
(825, 583)
(800, 533)
(800, 533)
(712, 800)
(240, 400)
(825, 570)
(800, 569)
(400, 400)
(825, 555)
(550, 366)
(800, 533)
(550, 365)
(825, 617)
(470, 800)
(591, 800)
(825, 549)
(550, 355)
(825, 530)
(550, 371)
(300, 400)
(550, 366)
(825, 546)
(550, 354)
(800, 533)
(550, 255)
(825, 583)
(800, 502)
(242, 800)
(550, 366)
(825, 549)
(550, 420)
(305, 399)
(712, 800)
(400, 400)
(800, 583)
(825, 599)
(800, 533)
(333, 800)
(677, 800)
(825, 601)
(533, 800)
(825, 621)
(550, 400)
(286, 400)
(533, 800)
(825, 549)
(286, 400)
(550, 418)
(564, 800)
(550, 363)
(787, 800)
(825, 657)
(295, 399)
(825, 549)
(825, 588)

(825, 588)
(550, 366)
(825, 549)
(413, 800)
(550, 359)
(550, 367)
(533, 800)
(609, 800)
(550, 366)
(550, 366)
(655, 800)
(572, 800)
(550, 366)
(704, 800)
(825, 549)
(825, 547)
(550, 366)
(550, 331)
(336, 800)
(282, 400)
(266, 400)
(550, 363)
(591, 800)
(550, 366)
(675, 800)
(550, 366)
(550, 387)
(825, 549)
(300, 400)
(550, 366)
(800, 571)
(800, 636)
(353, 400)
(800, 533)
(550, 367)
(550, 366)
(550, 366)
(545, 800)
(308, 400)
(550, 366)
(555, 799)
(200, 400)
(550, 363)
(550, 366)
(800, 564)
(800, 533)
(550, 344)
(550, 366)
(550, 391)
(571, 800)
(550, 366)
(534, 800)
(550, 366)
(800, 533)
(550, 378)
(550, 366)
(266, 400)
(550, 366)
(825, 623)
(825, 604)
(825, 618)
(533, 800)
(825, 549)
(298, 800)
(550, 357)
(422, 800)
(825, 549)
(550, 365)
(550, 355)
(417, 800)
(565, 800)
(825, 549)
(591, 799)
(283, 400)
(412, 800)
(534, 800)
(550, 366)
(825, 618)
(589, 800)
(600, 800)
(300, 400)
(825, 549)
(533, 800)
(825, 549)
(630, 800)
(294, 400)
(300, 400)
(550, 378)
(800, 800)
(642, 800)
(800, 600)

(825, 547)
(571, 800)
(800, 533)
(825, 551)
(400, 400)
(550, 365)
(825, 543)
(550, 366)
(550, 461)
(825, 549)
(300, 400)
(631, 800)
(825, 618)
(550, 366)
(550, 453)
(550, 367)
(800, 531)
(825, 497)
(825, 549)
(550, 453)
(550, 366)
(550, 351)
(800, 611)
(825, 533)
(550, 364)
(825, 549)
(523, 800)
(800, 515)
(550, 366)
(800, 535)
(551, 800)
(267, 400)
(825, 549)
(271, 400)
(550, 328)
(550, 384)
(555, 799)
(800, 547)
(825, 549)
(564, 800)
(550, 484)
(550, 366)
(550, 329)
(673, 800)
(550, 366)
(825, 549)
(825, 584)
(668, 800)
(286, 400)
(800, 533)
(800, 507)
(550, 395)
(550, 367)
(550, 366)
(550, 384)
(550, 416)
(375, 400)
(267, 400)
(600, 800)
(550, 366)
(550, 384)
(595, 800)
(550, 366)
(599, 800)
(800, 673)
(712, 800)
(582, 800)
(550, 366)
(550, 407)
(275, 400)
(825, 549)
(400, 400)
(240, 400)
(508, 800)
(550, 366)
(800, 532)
(825, 705)
(550, 366)
(550, 366)
(825, 630)
(550, 414)
(532, 800)
(550, 331)
(550, 366)
(400, 400)
(550, 366)
(800, 530)
(593, 800)
(548, 800)
(825, 549)
(800, 574)